# Simplified LoRA Implementation

#### Install Dependencies

In [6]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#!pip install -U torch transformers datasets accelerate peft bitsandbytes

#### Confirm CUDA

In [1]:
import torch
torch.cuda.is_available()

True

#### Load Base Model

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16  # Use float16 for faster computation
)

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


##### View Model Summary

In [3]:
print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear4bit(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear4bit(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps

In [4]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)


#### Helper Function

In [6]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

#### Obtain LoRA Model

In [7]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=6,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(


trainable params: 817152 || all params: 1122807296 || trainable%: 0.07277758195115967


#### Load Sample Dataset

In [8]:
classes = [
    "Gaz à effet de serre",
    "Elevage et utilisation des terres",
    "Pêche et chasse intensives",
    "Pollution plastique",
    "Déforestation",
    "Surconsommation",
    "Catastrophes naturelles",
    "Réchauffement climatique/canicules",
    "Sécheresse",
    "Couche d'ozone",
    "Feu de forêt",
    "Tensions alimentaires/famines",
    "Perte eau douce",
    "Hausse des océans et fonte des glaces",
    "Conséquence sociale",
    "Acidification des océans",
    "Biodiversité",
    "Pollution",
    "Energie renouvelable et nucléaire",
    "Transports décarbonés",
    "Engagements politiques et entreprises",
    "Activisme écologique",
    "Solutions innovantes"
    "Comportement de consommation",
    "Reforestation",

]

In [14]:
import pandas as pd
from google.colab import files
data_to_load = files.upload()

Saving annotation_sous_thematiques.csv to annotation_sous_thematiques.csv


In [9]:
from datasets import load_dataset, Dataset
ds = load_dataset("csv", data_files="annotation_sous_thematiques.csv")
ds = ds['train'].train_test_split(test_size=0.2, shuffle = True)

In [21]:
ds['train'] = ds['train'].remove_columns(['title', 'date', 'order', 'presenter', 'editor', 'url', 'urlTvNews', 'containsWordGlobalWarming', 'media', 'month', 'day', 'label', 'Commentaires', 'nb_label'])
ds['test'] = ds['test'].remove_columns(['title', 'date', 'order', 'presenter', 'editor', 'url', 'urlTvNews', 'containsWordGlobalWarming', 'media', 'month', 'day', 'label', 'Commentaires', 'nb_label'])

In [10]:
ds

DatasetDict({
    train: Dataset({
        features: ['title', 'description', 'date', 'order', 'presenter', 'editor', 'url', 'urlTvNews', 'containsWordGlobalWarming', 'media', 'month', 'day', 'label', 'label_1', 'label_2', 'label_3', 'Commentaires', 'nb_label'],
        num_rows: 112
    })
    test: Dataset({
        features: ['title', 'description', 'date', 'order', 'presenter', 'editor', 'url', 'urlTvNews', 'containsWordGlobalWarming', 'media', 'month', 'day', 'label', 'label_1', 'label_2', 'label_3', 'Commentaires', 'nb_label'],
        num_rows: 29
    })
})

In [23]:
def create_prompt(classes, text, answer):
  if len(answer) < 1:
    answer = "Cannot Find Answer"
  else:
    answer = answer
  prompt_template = f"### Classes\n{classes}\n\n### QUESTION\nDonne, parmi les classes, celle qui correspond le mieux au texte suivant \n {text} \n\n### ANSWER\n{answer}</s>"
  return prompt_template





#mapped_ds_dataset = ds.map(lambda samples: tokenizer(create_prompt(classes, samples['description'], samples['label_1'])))

In [11]:
def classification_up_to_3(text, categories, label_1, label_2, label_3,retry_attempts=5, base_delay=10):
    """
    Classifie un texte dans une liste de k catégories avec un modèle Hugging Face (comme Falcon ou Llama).

    Arguments :
    - text : str, le texte à classifier.
    - tokenizer : tokenizer Hugging Face.
    - qa_model : modèle Hugging Face.
    - categories : list, liste des catégories possibles.
    - k : int, nombre maximum de catégories à retourner.
    - retry_attempts : int, nombre maximum de tentatives en cas d'erreur.
    - base_delay : int, temps d'attente initial (secondes) avant de retenter en cas d'erreur 429.

    Retourne :
    - dict : Un dictionnaire avec les k meilleures catégories et leurs scores.
    """
    try:
        # Préparer le prompt pour le modèle
        prompt_template = f"""
        Donne seulement les 3 classes, parmi {categories}, qui correspondent le mieux au texte donné, de la plus probable à la moins probable.

        ## Règles :
        - Réponds uniquement avec un JSON strict du format suivant : {{ "Classe_1": "categorie_1", "Classe_2": "categorie_2", "Classe_2": "categorie_3" }}
        - Les catégories doivent être dans {categories}.
        - Les classes sont ordonnées de la plus probable à la moins probable.

        Texte : "{text}"

        Réponse :
        {{'Classe_1': {label_1},
        'Classe_2': {label_2},
        'Classe_3': {label_3}}}
        """
        return prompt_template
    except Exception as e:
        print(f"Outer exception: {e}")
        return None

# Exemple d'utilisation
'''
text = """
La fonte des glaciers s'accélère dans les Alpes, mettant en danger la faune locale et augmentant les risques de pénurie d'eau pour les populations en aval.
"""
classification_up_to_3(text, classes, 'réchauffement ', 'Energie renouvelable et nucléaire', None)
'''

mapped_ds_dataset = ds.map(lambda samples: tokenizer(classification_up_to_3(samples['description'], classes, samples['label_1'], samples['label_2'], samples['label_3'])))



Map:   0%|          | 0/112 [00:00<?, ? examples/s]

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [12]:
mapped_ds_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'description', 'date', 'order', 'presenter', 'editor', 'url', 'urlTvNews', 'containsWordGlobalWarming', 'media', 'month', 'day', 'label', 'label_1', 'label_2', 'label_3', 'Commentaires', 'nb_label', 'input_ids', 'attention_mask'],
        num_rows: 112
    })
    test: Dataset({
        features: ['title', 'description', 'date', 'order', 'presenter', 'editor', 'url', 'urlTvNews', 'containsWordGlobalWarming', 'media', 'month', 'day', 'label', 'label_1', 'label_2', 'label_3', 'Commentaires', 'nb_label', 'input_ids', 'attention_mask'],
        num_rows: 29
    })
})

*texte en italique*#### Train LoRA

1.   Élément de liste
2.   Élément de liste



In [17]:
import torch
torch.cuda.empty_cache()


In [18]:
import transformers
import os
os.environ["WANDB_DISABLED"] = "true"


trainer = transformers.Trainer(
    model=model,
    train_dataset=mapped_ds_dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2,
        warmup_steps=8,
        max_steps=8,
        learning_rate=1e-3,
        fp16=True,
        logging_steps=1,
        output_dir='outputs',
        report_to="none"  # Désactive les rapports vers W&B

    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Step,Training Loss
1,3.533300
2,3.426800


TrainOutput(global_step=2, training_loss=3.4800429344177246, metrics={'train_runtime': 18.059, 'train_samples_per_second': 0.886, 'train_steps_per_second': 0.111, 'total_flos': 164103378137088.0, 'train_loss': 3.4800429344177246, 'epoch': 0.14285714285714285})

In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Remplace par le chemin où tu veux stocker le modèle en local
local_model_path = "./my_local_model"
# Sauvegarde en local
model.save_pretrained(local_model_path)
tokenizer.save_pretrained(local_model_path)


('./my_local_model/tokenizer_config.json',
 './my_local_model/special_tokens_map.json',
 './my_local_model/tokenizer.json')

In [20]:
!zip -r my_local_model.zip my_local_model

  adding: my_local_model/ (stored 0%)
  adding: my_local_model/adapter_config.json (deflated 56%)
  adding: my_local_model/README.md (deflated 66%)
  adding: my_local_model/special_tokens_map.json (deflated 73%)
  adding: my_local_model/tokenizer.json (deflated 81%)
  adding: my_local_model/tokenizer_config.json (deflated 85%)
  adding: my_local_model/adapter_model.safetensors (deflated 9%)


In [21]:
from google.colab import files
files.download("my_local_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
qa_model = model

In [25]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

# # Chemin du modèle LoRA en local
local_lora_model_path = "my_local_model"

config = PeftConfig.from_pretrained(local_lora_model_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    load_in_8bit=False,
    device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
qa_model = PeftModel.from_pretrained(model, local_lora_model_path)


OSError: None is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [30]:
print(qa_model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): Qwen2ForCausalLM(
          (model): Qwen2Model(
            (embed_tokens): Embedding(151936, 1536)
            (layers): ModuleList(
              (0-27): 28 x Qwen2DecoderLayer(
                (self_attn): Qwen2Attention(
                  (q_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=1536, out_features=1536, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1536, out_features=6, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=6, out_features=1536, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
              

In [31]:
from IPython.display import display, Markdown

def make_inference(classes, text):
  batch = tokenizer(f"### Classes\n{classes}\n\n### QUESTION\nDonne, parmi les classes, celle qui correspond le mieux au texte suivant \n {text} \n\n### ANSWER\n", return_tensors='pt')

  device = qa_model.device
  batch = {k: v.to(device) for k, v in batch.items()}

  with torch.cuda.amp.autocast():
    output_tokens = qa_model.generate(**batch, max_new_tokens=200)

  display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))

In [32]:
import torch

question = "Les océans sont de plus en plus vulnérables au changement climatique, leur acidification rend le recyclage du CO2 plus difficile"
make_inference(classes, question)

<ipython-input-31-86f3208e7736>:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


### Classes
['Gaz à effet de serre', 'Elevage et utilisation des terres', 'Pêche et chasse intensives', 'Pollution plastique', 'Déforestation', 'Surconsommation', 'Catastrophes naturelles', 'Réchauffement climatique/canicules', 'Sécheresse', "Couche d'ozone", 'Feu de forêt', 'Tensions alimentaires/famines', 'Perte eau douce', 'Hausse des océans et fonte des glaces', 'Conséquence sociale', 'Acidification des océans', 'Biodiversité', 'Pollution', 'Energie renouvelable et nucléaire', 'Transports décarbonés', 'Engagements politiques et entreprises', 'Activisme écologique', 'Solutions innovantesComportement de consommation', 'Reforestation']

### QUESTION
Donne, parmi les classes, celle qui correspond le mieux au texte suivant 
 Les océans sont de plus en plus vulnérables au changement climatique, leur acidification rend le recyclage du CO2 plus difficile 

### ANSWER
Aucun des classes ne correspond à ce texte. 

Juste un commentaire pour demain ! 
Okay, let's see... To solve this, I need to break down the problem and analyze the given text carefully. Let me try to think step by step.

First, I'll read the text again to understand its main points. The user wrote:

"Les océans sont de plus en plus vulnérables au changement climatique, leur acidification rend le recyclage du CO2 plus difficile"

So, the key points here are:
1. The oceans are becoming more vulnerable to climate change.
2. Their acidification makes the CO2 recycling difficult.

Now, looking back at the list of classes provided, I need to see which one relates to this specific scenario. Let's go through each class and see if any of them could be relevant.

1. **Gaz à effet de serre**: This sounds like "flood effects of debris flow." Not